In [ ]:
#작업 2유형. 문제 지하철 탑승인원 수 예측하기 -회귀 문제 성능 지표 Mean Absolute Error
#기출 데이터 셋 독립변수 11개(날씨, 날짜 등), 종속변수 지하철 탑승인원수(정수형), train dataset 12000개, test dataset 2000개
#결측치 없음
#독립변수 중 name변수 지하철역명 삭제 시 감점처리 (40점 만점 중 30점 감점)

#DataFrame Setting 출처: 메타코드 
import numpy as np
import pandas as pd

np.random.seed(42)

n=1000 #데이터 수 

data = {
    'temperature': np.random.uniform(5, 35, n),         # 기온 (5도 ~ 35도)
    'humidity': np.random.uniform(20, 90, n),           # 습도 (20% ~ 90%)
    'precipitation': np.random.choice([0, 1], n, p=[0.7, 0.3]),  # 강수 유무
    'wind_speed': np.random.uniform(0, 10, n),          # 풍속 (0 ~ 10 m/s)
    'day_of_week': np.random.choice(range(7), n),       # 요일 (0: 일요일 ~ 6: 토요일)
    'is_holiday': np.random.choice([0, 1], n, p=[0.8, 0.2]),      # 공휴일 여부
    'is_weekend': np.random.choice([0, 1], n, p=[0.7, 0.3]),      # 주말 여부
    'hour': np.random.choice(range(24), n),             # 시간대 (0시 ~ 23시)
    'school_vacation': np.random.choice([0, 1], n, p=[0.8, 0.2]), # 방학 여부
    'special_event': np.random.choice([0, 1], n, p=[0.95, 0.05])  # 특별 행사 여부
}

# passenger_count 계산 (가중치 + 노이즈 포함)
data['passenger_count'] = (
    300
    + 5 * data['temperature']
    - 2 * data['humidity']
    + 50 * data['precipitation']
    - 10 * data['wind_speed']
    - 20 * data['is_holiday']
    + 15 * data['is_weekend']
    + 3 * data['hour']
    - 50 * data['school_vacation']
    + 80 * data['special_event']
    + np.random.normal(0, 30, n)  # 평균 0, 표준편차 30인 노이즈
)

#종속변수는 정수형으로 변환 
data['passenger_count'] = np.round(data['passenger_count']).astype(int)

#데이터프레임으로 변환
df=pd.DataFrame(data)

#독립변수 중에서 소수점이 있는 변수는 소수점 첫째자리까지 반올림
df['temperature'] =np.round(df['temperature'],1)
df['humidity'] =np.round(df['humidity'],1)
df['wind_speed'] =np.round(df['wind_speed'],1)

station_names = [
    '시청', '을지로입구', '을지로3가', '을지로4가', '동대문역사문화공원', '신당', '상왕십리', '왕십리', '한양대', '뚝섬',
    '성수', '건대입구', '구의', '강변', '잠실나루', '잠실', '잠실새내', '종합운동장', '삼성', '선릉',
    '역삼', '강남', '교대', '서초', '방배', '사당', '낙성대', '서울대입구', '봉천', '신림',
    '신대방', '구로디지털단지', '대림', '신도림', '도림천', '양천구청', '신정네거리', '신정', '까치산', '까치산',
    '대방', '노량진', '용산', '이촌', '동작', '총신대입구', '사당', '남태령', '방배', '교대',
    '강남', '역삼'
]

name_variable=[station_names[i%len(station_names)]for i in range(1000)]

#'name'변수 추가
df['name' ] =name_variable

#범주형 데이터를 카테고리형으로 변환 precipitation 강수량
categorical_columns=['precipitation', 'day_of_week', 'is_holiday', 'is_weekend']

#카테고리형으로 변환 
df[categorical_columns]=df[categorical_columns].astype('category')

#Train,Test셋으로 나누기
train = df.iloc[:700] #초기 700개를 train으로 나누기
test = df.iloc[700:] #마지막 300개를 test로 나누기
y_test = test['passenger_count'] #실제 시험에서 주어지지 않기 때문에 직접 작성한다.
test= test.drop(['passenger_count'], axis=1) #passenger_count 지하철 탑승 인원
train.head(3)


,temperature,humidity,precipitation,wind_speed,day_of_week,is_holiday,is_weekend,hour,school_vacation,special_event,passenger_count,name
0,16.2,33.0,0,6.7,1,1,0,1,0,0,215,시청
1,33.5,57.9,0,8.0,4,0,0,22,0,0,328,을지로입구
2,27.0,81.1,1,2.5,4,0,1,21,0,0,383,을지로3가


In [28]:
#1. 데이터 준비 및 분리
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# train, test 데이터가 주어졌다고 가정
x_train = train.drop(['passenger_count'], axis=1)
y_train = train['passenger_count']
x_test = test

#2. 데이터 탐색 EDA
# 기본 구조 확인
print(x_train.shape, x_test.shape, y_train.shape)

# 상위 데이터 확인
print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

# 데이터 타입 및 결측치 확인
print(x_train.info())
print(x_test.info())
print(y_train.info())

# 기초통계량 확인
print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

# object, category 확인
print(x_train.describe(include='object'))
print(x_test.describe(include='object'))
print(x_train.describe(include='category'))
print(x_test.describe(include='category'))

#3. 데이터 전처리 (전처리/결측치 처리 및 인코딩)
# 결측치 확인
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

# 이상치 처리: y값이 음수인 경우 0으로 수정
y_train = y_train.copy()
y_train[y_train < 0] = 0

# One-Hot Encoding (더미화)
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)

# 열 정렬 및 맞추기
x_train, x_test = x_train.align(x_test, join='left', axis=1, fill_value=0)

#4. 회귀모델 학습 및 검증(MAE 평가)
# 검증을 위한 train/val 분리
x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# 모델 선언 및 학습
model = LinearRegression()
model.fit(x_tr, y_tr)

# 검증 데이터 예측 및 MAE 산출
val_pred = model.predict(x_val)
mae = mean_absolute_error(y_val, val_pred)
print("MAE:", mae)

#5. 테스트 데이터 예측 및 결과 저장
# 전체 학습 데이터로 다시 학습
model.fit(x_train, y_train)

# 테스트 데이터 예측
pred = model.predict(x_test)

# 음수 방지 처리
pred = np.clip(pred, 0, None)

# 예측값 저장
submission = pd.DataFrame({'pred': pred})
submission.to_csv('submission.csv', index=False)



(700, 11) (300, 11) (700,)
   temperature  humidity precipitation  wind_speed day_of_week is_holiday  \
0         16.2      33.0             0         6.7           1          1   
1         33.5      57.9             0         8.0           4          0   
2         27.0      81.1             1         2.5           4          0   

  is_weekend  hour  school_vacation  special_event   name  
0          0     1                0              0     시청  
1          0    22                0              0  을지로입구  
2          1    21                0              0  을지로3가  
     temperature  humidity precipitation  wind_speed day_of_week is_holiday  \
700         21.0      87.6             0         6.1           0          0   
701          6.6      50.3             1         6.8           4          0   
702         15.1      41.8             0         3.2           3          1   

    is_weekend  hour  school_vacation  special_event name  
700          0    18                0          